# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [ ]:
!pip install gmaps

In [ ]:
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
output_data = pd.read_csv('../output_data/cities.csv')
output_data
#output_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     548 non-null    int64  
 1   City        548 non-null    object 
 2   Cloudiness  548 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        548 non-null    int64  
 5   Humidity    548 non-null    int64  
 6   Lat         548 non-null    float64
 7   Lng         548 non-null    float64
 8   Max Temp    548 non-null    float64
 9   Wind Speed  548 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 42.9+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
#Use the Lat and Lng as locations and Humidity as the weight.
output_data = output_data.dropna()
locations = output_data[["Lat","Lng"]].astype(float)
weight = output_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations=locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add the layer
fig.add_layer(heat_layer)

# show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
perfectcity_df= output_data.loc[(output_data['Max Temp']<70)&(output_data['Max Temp']>60)&(output_data['Wind Speed']<10)&(output_data['Cloudiness']!=5)].dropna()
perfectcity_df.head

<bound method NDFrame.head of      City_ID               City  Cloudiness Country        Date  Humidity  \
9          9  pushkinskiye gory           6      RU  1558378756        70   
22        22              ambon          11      FR  1558378758        93   
37        37            varhaug           0      NO  1558378761        63   
48        48        port alfred          67      ZA  1558378763        69   
69        69            fortuna          20      ES  1558378767        77   
..       ...                ...         ...     ...         ...       ...   
514      514             loikaw         100      MM  1558378892        85   
522      522          raudeberg          20      NO  1558378894        76   
533      533            hakkari           0      FI  1558378896        46   
538      538   la seyne-sur-mer           0      FR  1558378897        72   
545      545            veraval           0      FR  1558378898        62   

       Lat    Lng  Max Temp  Wind Speed  
9  

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Set df
hotel_df= perfectcity_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df["Hotel Name"]= ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,pushkinskiye gory,RU,57.02,28.91,
22,ambon,FR,47.55,-2.56,
37,varhaug,NO,58.61,5.65,
48,port alfred,ZA,-33.59,26.89,
69,fortuna,ES,38.18,-1.13,
...,...,...,...,...,...
514,loikaw,MM,19.67,97.21,
522,raudeberg,NO,61.99,5.14,
533,hakkari,FI,61.34,23.75,
538,la seyne-sur-mer,FR,43.10,5.88,


In [17]:
# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng= row['Lng']
    params['location']= f'{lat}, {lng}'
    
# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# run request using the params dictionary above 
    response = requests.get(base_url, params=params)
    address = response.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name']= address['results'][0]['name']
    except:
         print("no data 4 u")

hotel_df.head

no data 4 u
no data 4 u


,City,Country,Lat,Lng,Hotel Name
9,pushkinskiye gory,RU,57.02,28.91,"The recreation center ""Pushkinogorye"""
22,ambon,FR,47.55,-2.56,Hotel de la Plage - Damgan - Morbihan - Bretagne
37,varhaug,NO,58.61,5.65,Jonas Bue
48,port alfred,ZA,-33.59,26.89,The Halyards Hotel
69,fortuna,ES,38.18,-1.13,Balneario de Leana


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))